# Create crossval tables

This notebook is dedicated to create the cv table to all tunings produced during 2020 for tracking purpose.


In [1]:
from core import crossval_table, get_color_fader
from Gaugi import mkdir_p
from copy import copy
from pprint import pprint
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt

import atlas_mpl_style as ampl
ampl.use_atlas_style()

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...


In [2]:
mkdir_p('output/crossval')
mkdir_p('output/training')

In [13]:
def create_op_dict(op, decoration='reference'):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : decoration+"/"+op+"/pd_ref#0",
              'fa_ref'    : decoration+"/"+op+"/fa_ref#0",
              'sp_ref'    : decoration+"/"+op+"/sp_ref",
              'pd_val'    : decoration+"/"+op+"/pd_val#0",
              'fa_val'    : decoration+"/"+op+"/fa_val#0",
              'sp_val'    : decoration+"/"+op+"/sp_val",
              'pd_op'     : decoration+"/"+op+"/pd_op#0",
              'fa_op'     : decoration+"/"+op+"/fa_op#0",
              'sp_op'     : decoration+"/"+op+"/sp_op",

              # Counts
              'pd_ref_passed'    : decoration+"/"+op+"/pd_ref#1",
              'fa_ref_passed'    : decoration+"/"+op+"/fa_ref#1",
              'pd_ref_total'     : decoration+"/"+op+"/pd_ref#2",
              'fa_ref_total'     : decoration+"/"+op+"/fa_ref#2",
              'pd_val_passed'    : decoration+"/"+op+"/pd_val#1",
              'fa_val_passed'    : decoration+"/"+op+"/fa_val#1",
              'pd_val_total'     : decoration+"/"+op+"/pd_val#2",
              'fa_val_total'     : decoration+"/"+op+"/fa_val#2",
              'pd_op_passed'     : decoration+"/"+op+"/pd_op#1",
              'fa_op_passed'     : decoration+"/"+op+"/fa_op#1",
              'pd_op_total'      : decoration+"/"+op+"/pd_op#2",
              'fa_op_total'      : decoration+"/"+op+"/fa_op#2",
    })
    return d


op_names = ['loosetrk']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op, "reference")

In [14]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [15]:
cv_v3_el  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [16]:
# just for comparison
cv_v3_el.fill(  '/home/jodafons/public/cern_data/tunings/r0/Zee/v3_el/*/*/*.npz' , 'v3-el' )

Reading v3-el...: 100%|██████████| 1250/1250 [01:54<00:00, 10.92it/s]

2022-03-25 23:19:06,983 | Py.crossval_table                       INFO Reading file for v3-el tag from /home/jodafons/public/cern_data/tunings/r0/Zee/v3_el/*/*/*.npz
2022-03-25 23:19:06,984 | Py.crossval_table                       INFO There are 1250 files for this task...
2022-03-25 23:19:06,984 | Py.crossval_table                       INFO Filling the table... 
2022-03-25 23:21:01,502 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


In [19]:
cv_v3_el.to_csv('output/crossval/table_v3_el.csv')

### 1.1) Get best inits and sorts:

In [20]:
best_inits = cv_v3_el.filter_inits("max_sp_val")
best_inits = best_inits.loc[best_inits.model_idx==0]
best_sorts = cv_v3_el.filter_sorts( best_inits , 'max_sp_op')

### 1.2) Make the table:

In [22]:
for op in op_names:
    cv_v3_el.dump_beamer_table( best_inits.loc[best_inits.op_name == op] ,  
                            'tuning_v3_el'+op, 
                             title = op+' Tunings (v3-el)', 
                             tags = ['v3-el']
                           )

2022-03-25 23:23:37,204 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v3_elloosetrk.pdf latex code...


In [23]:
!mv *.tex output/crossval
!mv *.pdf output/crossval